# Les références, l'emprunt et les pointeurs

---

> Tout comme en C / C++, il existe en Rust les notions de pointeurs et de références. Cependant, Rust diffère quelque peu de ces langages quant à leur gestion. En effet, grave aux mécanismes de durée de vie et d'initialisation des variables, **il est impossible d'avoir des références nulles**. Les références et les pointeurs sont toujours en rapport avec les adresses mémoire, mais leur gestion, et disons le, bien plus transparente que dans des langages plus bas niveau.

> Afin de pouvoir coder une FFI efficace, Rust permet aussi l'utilisation des pointeurs façon C, ils sont appelés **raw pointers** ou pointeurs bruts, et font partie du monde de l'unsafe Rust.

## Les références

> Les références servent avant tout a **ne pas transférer l'ownership des variables aux fonctions** que nous appelons. Lors de l'appel à la macro dbg! par exemple, les programmeurs mettent souvent, comme paramètre, le symbole `&` avant le nom de la variable, ceci évite tout simplement de « perdre » cette variable pour un vulgaire debug.
```
let mut a: usize = 42;
dbg!(&a); // Utilisation du référencement, l'adresse de `a` est envoyée a dbg! et non la variable `a` elle-même.
a += 1; // La variable `a` est toujours accessible ici.
```

> Nous avons déjà croisé des références comme `&self` ou `&mut self` lors de l'écriture des méthodes des structures.

### Le référencement

> Bien que le terme **référencement** n'est pas trop d'usage dans la communauté Rust, je trouve qu'il a du sens pour les débutants puisqu'il est le contraire du **deréférencement**.

> Le référencement est tout simplement la création d'une référence.

- Voici un exemple d'utilisation des références en C :
```
void ma_fonction(int *x)   // Utilisation du symbole *
{
    printf("La valeur de x est %i\n", *x);
    *x += 1;
    printf("La nouvelle valeur de x est %i\n", *x);
}
int main()
{
    int x = 10;
    ma_fonction(&x);         // Utilisation du symbole &
    printf("Apres le retour de la fonction, la valeur de x est %i\n", x);
}
```
> En C, on utilise les pointeurs et les références non pas pour éviter le transfert d'ownership, notion qui n'existe pas dans ce langage, mais principalement pour que les fonctions appelées puissent modifier les variables passées en paramètre et/ou éviter une trop grosse surcharge de la pile ou pour d'autres raisons encore...

- Ce qui change en Rust c'est que sur le prototype de la fonction appelée, on n'utilisera pas le symbole `*` mais le symbole de référence `&` : 

In [18]:
fn ma_fonction(x: &mut usize) {           // Utilisation du symbole & en Rust
    println!("La valeur de x est {}", x); // x ou *x ne fait pas de difference pour println!
    *x += 1; // Comme en C, la modification de x passe par l'emploi de l'operateur de dereferencement *
    println!("La nouvelle valeur de x est {}", *x);
}
fn main() {
    let mut x: usize = 10;
    ma_fonction(&mut x); // Passage d'une reference mutable puisque la fonction va modifier x
    println!("Apres le retour de la fonction, la valeur de x est {}", x);
}
main()

La valeur de x est 10
La nouvelle valeur de x est 11
Apres le retour de la fonction, la valeur de x est 11


()

> Ce code en Rust est strictement équivalent au code C plus haut.

### Le dereferencement

> Dereferencer une variable consiste a accéder à sa valeur derrière la référence. On utilisera le symbole de dereferencement `*`.

- L'opérateur de derefencement va s'utiliser par exemple lors d'opérations entres les références d'un type et le type lui-même. À quelques exceptions près, Rust ne peut pas comparer deux types différents : *ref mon_type n'est pas la même que mon_type tout court* :

In [20]:
#[derive(Debug, Eq, PartialEq)]
struct S {
    i: usize,
    j: usize,
}
fn check_struct(s1: &S) -> bool { // Cette fonction retourne un bool apres comparaison de la structure en param
    s1 == S {
        i: 12,
        j: 42,
    }
}
dbg!(check_struct(&S {
    i: 12,
    j: 42,
}))

Error: can't compare `&S` with `S`

- La code deviendra valide si l'on accepte de dereferencer s1 dans la fonction check_struct :

In [24]:
#[derive(Debug, Eq, PartialEq)]
struct S {
    i: usize,
    j: usize,
}
fn check_struct(s1: &S) -> bool { // Cette fonction retourne un bool apres comparaison de la structure en param
    *s1 == S { // Utilisation de l'operateur de dereferencement pour retrouver le type S et non &S sur s1.
        i: 12,
        j: 42,
    }
}
dbg!(check_struct(&S {
    i: 12,
    j: 42,
}))

[src/lib.rs:43] check_struct(&S { i: 12, j: 42 }) = true


true

> Il est possible aussi de faire le contraire, et de référencer le second terme sois S ici. **s1 == &S** fonctionne très bien !

- On peut tres bien comparer des références de référence de référence etc... :

In [25]:
{    
    let a = 16;
    let b = 16;
    let ra = &a;
    let rb = &b;
    assert_eq!(ra, rb);
    assert_eq!(*ra, *rb);
    assert_eq!(&ra, &rb);
    assert_eq!(&&ra, &&rb);

    // Ne compile pas: comparaison entre &integer et integer
    // assert_eq!(*ra, rb);
}

()

## Les références implicites

- Prenons l'exemple suivant :

In [28]:
{
    #[derive(Debug, Copy, Clone)]
    struct Vector {
        i: i32,
        j: i32,
    }

    impl Vector {
        fn add_assign(&mut self, other: Self) {
            self.i += other.i;
            self.j += other.j;
        }
    }

    let mut v = Vector{i: 10, j: -3};
    dbg!(v);
    v.add_assign(Vector{i: 2, j: 4});
    dbg!(v);
}

[src/lib.rs:170] v = Vector {
    i: 10,
    j: -3,
}
[src/lib.rs:172] v = Vector {
    i: 12,
    j: 1,
}


()

> La méthode add_assign prend une référence mutable de Vector pour self, pourtant, on a envoyé un type Vector et non une référence de Vector. **L’opérateur `.` retourne implicitement la référence (mutable ici) de v**.
```
v.add_assign(Vector{i: 2, j: 4})
```
> est strictement equivalent a :
```
(&mut v).add_assign(Vector{i: 2, j: 4});
```
**On dit que l’opérateur `.` emprunte (borrowing) ici une référence mutable de v.**

## Les références : mutabilité et exclusivité

> Il ne peut pas y avoir de référence Null en Rust. Le compilateur s'engage à vérifier si la donnée référencée est toujours 'en vie' cf. notion de lifetime et qui sont ceux qui l'ont emprunté cf. notions d'ownership et de borrowing.

> **VOICI LES REGLES :**

> - **Si j'ai une référence mutable sur une variable, l'accés à cette derniere ne peut se faire que par cette meme référence, et cela tant que la référence est en vie** (cf: lifetime). Cela permet surtout d'éviter des bugs si la variable ainsi référencée venait à etre changee au nez et à la barbe de la référence. 

>  - **Si j'ai une référence non mutable sur une variable qui peut être mutée, que ce soit directement ou par une référence mutable, je ne peux pas modifier les données de cette variable tant que ma référence non mutable est en vie.**

> *Par contre, si j'ai un type non mutable, je peux avoir des dizaines de références dessus sans que cela pose un problème puisque les données resteront dans tous les cas constantes.*

In [29]:
{
    let mut num = 5;
    let _num_ref = &mut num;
}

()

> En langue Rustienne, on dit que l'espace mémoire de num a été emprunté de façon mutable par _num_ref. **mutably borrowed**

> **NB** C'est sans doute, parmi les aspects de Rust, un des plus difficiles à vraiment saisir...

### L'application de la première règle

- Prenons par exemple ce bout de code :

In [2]:
{
    let mut num = 5;
    let num_ref = &mut num;
    dbg!(num);
    *num_ref = 6; // la ref mutable `num_ref` etait toujours en vie puisque je l'utilise ici.
}

Error: cannot use `num` because it was mutably borrowed

- Ici, après avoir déclaré l'entier num, on donne la garantie de l’exclusivité sur l'espace mémoire de num a la référence mutable num_ref :
```
    let mut num = 5;
    let num_ref = &mut num;
```
- Puis on tente d’accéder (pourtant seulement en lecture) a num....
```
dbg!(num);
```
- ... alors que la référence mutable num_ref vit toujours ici, à la fin du code.
```
    *num_ref = 6;
```

- J'ai deux facons de corriger ce code :

#### L'acces exclusif via ma référence mutable

- Je respecte la règle et je n'accède a la variable seulement via la référence mutable num_ref :

In [40]:
{
    let mut num = 5;
    let num_ref = &mut num;
    dbg!(*num_ref); // J'utilise ma reference mutable pour acceder a la variable. 
    *num_ref = 6;
}

[src/lib.rs:162] *num_ref = 5


()

> On peut généraliser cela à tous les cas de figure, dès que l'on a une référence mutable sur quelque chose, l’exclusivité doit être garantie. Que ce soit vis-à-vis d'autres références mutables ou même non-mutables !

- Le code n'aurait pas fonctionné non plus si j'avais tenté un accès via une autre référence, non mutable cette fois:

In [3]:
{
    let mut num = 5;
    let imutable_ref = &num;
    let num_ref = &mut num;
    dbg!(*imutable_ref); // J'utilise ma reference mutable pour acceder a la variable. 
    *num_ref = 6; // num_ref est toujours en vie !
}

Error: cannot borrow `num` as mutable because it is also borrowed as immutable

#### Laisser mourir ma référence mutable

- Le comportement ne serait pas le même si l'on avait plus tenté d’accéder a num_ref à la fin du code. Pour le compilateur, la référence mutable serait oubliée ou morte.

In [4]:
{
    let mut num = 5;
    let num_ref = &mut num;
    dbg!(num); // la reference num_ref n'etant pas utilisee dans la suite du code, elle n'existe deja plus ! 
    // *num_ref = 6;
}

[src/lib.rs:25] num = 5


()

### L'application de la seconde règle

- J'ai une variable mutable et une référence non mutable dessus :

In [6]:
{
    let mut num = 5;
    let num_ref = &num; // reference non mutable
    num += 1; // Modification de la variable au nez et a la barbe de la reference non mutable num_ref ...
    dbg!(num_ref); // La reference non-mutable est toujours en vie !
    dbg!(num);
}

Error: cannot assign to `num` because it is borrowed

- La solution consiste à ne pas modifier ma variable tant que la référence non mutable vit :

In [10]:
{
    let mut num = 5;
    let num_ref = &num; // reference non mutable
    dbg!(num_ref);
    num += 1; // la reference num_ref n'existe plus parce que non utilisee dans le code qui suit. je peux donc.
    dbg!(num);
}

[src/lib.rs:25] num_ref = 5
[src/lib.rs:27] num = 6


()

> Tant qu'une référence non mutable est vivante, l'objet ne peut être modifié par aucun moyen (sauf code unsafe)

### Et si rien n'est mutable

- Cependant si je n'ai que des références non-mutables sur un type non mutable, la règle ne s'applique pas, et je peux en avoir plusieurs :

In [38]:
{
    let num = 5;
    let imutable_ref_1 = &num;
    let imutable_ref_2 = &num;
    let imutable_ref_3 = imutable_ref_2;
    dbg!(imutable_ref_1);
    dbg!(imutable_ref_2);
    dbg!(imutable_ref_3);
    dbg!(num);
}

[src/lib.rs:164] imutable_ref_1 = 5
[src/lib.rs:165] imutable_ref_2 = 5
[src/lib.rs:166] imutable_ref_3 = 5
[src/lib.rs:167] num = 5


()

### Les structures et leurs membres

- Enfin, il est important de souligner que dans le cas d'une structure, on peut avoir des références mutables sur chacun de ses membres, du moins tant qu'il n'y a pas de référence sur la structure elle-même :

In [44]:
{
    #[derive(Debug)]
    struct S {
        i: isize,
        j: isize,
    }
    let mut s = S {
        i: 3,
        j: 4,
    };
    let ref_i = &mut s.i; // Une reference mutable sur le membre i
    let ref_j = &mut s.j; // Une reference mutable sur le membre j
    *ref_i = -11;
    *ref_j = -12;
    // dbg!(s);           // Il y a encore des references mutables, donc ca je ne peux pas faire...
    *ref_i = -21;
    *ref_j = -22;
    dbg!(s);              // Plus de reference mutable ici ne vit, je peux acceder a ma structure.
}

[src/lib.rs:176] s = S {
    i: -21,
    j: -22,
}


()

- Par contre, si je cherche à emprunter une référence mutable sur ma structure entière après avoir déjà emprunté un de ses membres, les règles d'exclusivité s'appliqueront et mon programme ne pourra pas compiler :

In [45]:
{
    #[derive(Debug)]
    struct S {
        i: isize,
        j: isize,
    }
    let mut s = S {
        i: 3,
        j: 4,
    };
    let ref_i = &mut s.i; // Une reference mutable sur le membre i
    let ref_s = &mut s;
    *ref_i = -11;
}

Error: unused variable: `ref_s`

Error: cannot borrow `s` as mutable more than once at a time

**NB** *Il est possible de tricher un peu avec ces règles en utilisant le type `RefCell`, nous en parlerons dans un prochain chapitre.*

> **N'hesitez pas si vous avez des questions, l'exclusivité des références mutables est un des points des plus délicats à saisir.**

## Les raw pointers

- Pour ceux qui voudront faire un peu de FFI avec du code C ou C++, voici un exemple d'utilisation des raw pointeurs en Rust :

In [58]:
    let mut a: u32 = 42;
    let ptr: *const u32 = &mut a;                // Creation d'un raw pointer sur a : Contient l'adresse de a
    dbg!(ptr);            // Addresse memoire de a
    // Simple, Basique
    unsafe {
        dbg!(*ptr);       // Contenu de a : unsafe est necessaire ici
    }
    let prim: [u32; 6] = [2, 3, 5, 7, 11, 13];
    let ptr: *const u32 = prim.as_ptr();          // Creation d'un raw pointer sur le tableau
    // Le programmeur sait ce qu'il fait ici ...
    unsafe {
        dbg!(*ptr);
        dbg!(*ptr.offset(1));                  // La methode offset modifie l'addresse relativement au pointeur
        dbg!(*ptr.offset(2));
        dbg!(*ptr.offset(3));
        dbg!(*ptr.offset(4));
        dbg!(*ptr.offset(5));
    }
    // ... Ah... Il vient de craquer !
    unsafe {
        dbg!(*ptr.offset(66224411));
    }

[src/lib.rs:150] ptr = 0x00007fff71b55104
[src/lib.rs:154] *ptr = 42
[src/lib.rs:163] *ptr = 2
[src/lib.rs:164] *ptr.offset(1) = 3
[src/lib.rs:165] *ptr.offset(2) = 5
[src/lib.rs:167] *ptr.offset(4) = 11
[src/lib.rs:166] *ptr.offset(3) = 7
[src/lib.rs:168] *ptr.offset(5) = 13
Segmentation fault.
   0: evcxr::runtime::Runtime::install_crash_handlers::segfault_handler
   1: <unknown>
   2: <core::panic::unwind_safe::AssertUnwindSafe<F> as core::ops::function::FnOnce<()>>::call_once
   3: run_user_code_34
   4: evcxr::runtime::Runtime::run_loop
   5: evcxr::runtime::runtime_hook
   6: evcxr_jupyter::main
   7: std::sys_common::backtrace::__rust_begin_short_backtrace
   8: std::rt::lang_start::{{closure}}
   9: core::ops::function::impls::<impl core::ops::function::FnOnce<A> for &F>::call_once
             at /rustc/8460ca823e8367a30dda430efda790588b8c84d3/library/core/src/ops/function.rs:287:13
      std::panicking::try::do_call
             at /rustc/8460ca823e8367a30dda430efda790588b8c8

Error: Subprocess terminated with status: signal: 6 (SIGABRT) (core dumped)

> En effet, la dernière instruction n'est pas ouf :p

> **NB** En cas de crash d'un programme, les parties de code notées **unsafe** doivent être regardées en premier, les erreurs viennent de là.

## Exercices

### Quel est le problème dans ce bout de code ?

```
fn example() -> () {
    #[derive(Debug)]
    struct MyStruct {
        i: usize,
        j: usize,
    }
    let s = MyStruct {
        i: 0,
        j: 1,
    };
    dbg!(s);
    dbg!(s.i);
}
```

> Lors de l'appel a l'avant-dernier dbg! Il aurait fallu passer une référence de la struct s plutôt que s lui-même. En effet, la structure n'étant pas Copy, elle a été "moved" lors de l'appel de la macro dbg!. On y a donc plus accès lors du dernier appel a dbg!.
```
        j: 1,
    };
    dbg!(&s);
    dbg!(s.i);
}
```

### Ce code compile-t-il ? Son comportement, est-il logique ?


```
fn increment_number(mut i: usize) {
    i += 1;
}
fn example() -> () {
    let mut i: usize = 42;
    increment_number(i);
    println!("La nouvelle valeur de i est {}", i);
}
example();
```

> Oui, ce code compile correctement. Cependant, lors de l'appel à la fonction increment_number, la variable i de type usage est copiée implicitement (le type implémente le trait Copy), ainsi la variable i de la fonction exemple n'est jamais incrémentée, c'est sa copie qui l'a été !

> à la place, il aurait fallu un code du genre :
```
fn increment_number(i: &mut usize) { // Reference mutable de usize
    *i += 1;
}
fn example() -> () {
    let mut i: usize = 42;
    increment_number(&mut i);        // On passe une reference mutable
    println!("La nouvelle valeur de i est {}", i);
}
example();
```

### Ai-je bien respecter toutes les règles de Rust dans le code suivant ?

```
{
    #[derive(Debug)]
    struct MyStruct(u8);
    impl MyStruct {
        fn increment_inner(&mut self) {
            self.0 += 1;
        }
    }
    fn example() {
        let mut s = MyStruct(41);
        let ref_s = &s;
        s.increment_inner();
        dbg!(ref_s);
    }
    example();
}
```

> La méthode increment_inner de l'implémentation de MyStruct prend une référence mutable de self. Or, je viens déjà de récupérer une référence immuable de la structure juste avant de l'appeler. Le compilateur ne veut pas que j'emprunte ma structure de façon mutable puisque je l'ai déjà empruntée comme immuable.
```
[E0502] Error: cannot borrow `s` as mutable because it is also borrowed as immutable
    ╭─[command_59:1:1]
    │
 11 │         let ref_s = &s;
    ·                     ─┬  
    ·                      ╰── immutable borrow occurs here
 12 │         s.increment_inner();
    ·         ─────────┬─────────  
    ·                  ╰─────────── mutable borrow occurs here
 13 │         dbg!(ref_s);
    ·              ──┬──  
    ·                ╰──── immutable borrow later used here
────╯
```

### Que peut-on dire de ce code-là ?

```
let ptr: *const u8 = std::ptr::null();
unsafe {
    dbg!(*ptr);
}
```

> C'est unsafe et ça segfault. Pointeur nul ! Code nul ! Zéro !